In [1]:
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import os

In [2]:
IMG_SIZE = (128, 128)
BATCH_SIZE = 16
DATASET_DIR = "images"

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels='inferred',
    label_mode='int',
    class_names=['spiral', 'elliptical'],
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    labels='inferred',
    label_mode='int',
    class_names=['spiral', 'elliptical'],
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)
# Store label names
class_names = ['spiral', 'elliptical']
print("Classes:", class_names)

Found 153 files belonging to 2 classes.
Using 123 files for training.
Found 153 files belonging to 2 classes.
Using 30 files for validation.
Classes: ['spiral', 'elliptical']


In [4]:
# Build CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

C:\Users\tucke\anaconda3\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
# Compile model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [6]:
# Train model
history = model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 139ms/step - accuracy: 0.5784 - loss: 0.7106 - val_accuracy: 0.8000 - val_loss: 0.6505
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - accuracy: 0.8382 - loss: 0.6066 - val_accuracy: 0.8000 - val_loss: 0.4769
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.9079 - loss: 0.3371 - val_accuracy: 0.7333 - val_loss: 0.6686
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.8050 - loss: 0.4706 - val_accuracy: 0.8333 - val_loss: 0.5182
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.9413 - loss: 0.1474 - val_accuracy: 0.8000 - val_loss: 0.5627
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.9334 - loss: 0.1880 - val_accuracy: 0.7667 - val_loss: 0.7886
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.9457 - loss: 0.1642 - val_accuracy: 0.7667 - val_loss: 0.6504
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.9929 - loss: 0.0643 - val_accuracy: 0.8000 - val_loss: 0.

In [7]:
final_train_loss, final_train_acc = model.evaluate(train_ds)
final_val_loss, final_val_acc = model.evaluate(val_ds)

#The closer/higher the 2 Accuracies, the better
print(f"Final Training Accuracy: {final_train_acc * 100:.2f}%")
print(f"Final Validation Accuracy: {final_val_acc * 100:.2f}%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9300 - loss: 0.1437
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.7819 - loss: 0.7658
Final Training Accuracy: 95.12%
Final Validation Accuracy: 76.67%


In [8]:
def predict_image(image_path, threshold=0.6):
    img = load_img(image_path, target_size=IMG_SIZE)
    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis
    predictions = model.predict(img_array)[0]

    # Get prediction and confidance
    top_index = np.argmax(predictions)
    top_confidence = predictions[top_index]
    predicted_class = class_names[top_index]

    # Irregular if not confidant
    if top_confidence < threshold:
        predicted_class = 'irregular'

    print(f"Prediction: {predicted_class} (Confidence: {top_confidence:.2f})")
    return predicted_class

In [9]:
predict_image("test/988001.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Prediction: elliptical (Confidence: 0.93)


'elliptical'

In [10]:
predict_image("test/987261.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Prediction: spiral (Confidence: 0.99)


'spiral'

In [11]:
predict_image("test/100150.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Prediction: elliptical (Confidence: 0.75)


'elliptical'

In [12]:
predict_image("test/101007.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction: spiral (Confidence: 1.00)


'spiral'

In [13]:
predict_image("test/101623.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Prediction: elliptical (Confidence: 0.87)


'elliptical'

In [14]:
predict_image("test/100765.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Prediction: spiral (Confidence: 1.00)


'spiral'

In [15]:
model.save("galaxy_model.keras")

In [16]:
predict_image("test_flask_image.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Prediction: spiral (Confidence: 0.99)


'spiral'

In [17]:
img = load_img("test_flask_image.jpg", target_size=(128, 128))
img = img.convert("RGB")
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

model.predict(img_array)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


array([[0.99491704, 0.00508303]], dtype=float32)